ELECTRICITY

In [1]:
# -------------------------------------------------------------
#  Google Colab ‑ single‑cell script
#  Downloads the ADB electricity dataset, then
#    ①  builds ONE *merged* CSV (8 columns)
#    ②  writes ONE folder / indicator with its own raw+clean CSVs
#  Folder layout (everything overwritten on each run):
#
#  MyDrive/ADB/Global/Electricity
#  ├── Merged_Indicators
#  │   ├── raw/
#  │   │   └── electricity_indicators_raw.<csv|xlsx>
#  │   └── processed/
#  │       └── electricity_indicators_merged_pivot.csv   <-- 8 cols
#  └── Individual_Indicators
#      └── <slug‑per‑indicator>/
#          ├── raw/      (<indicator>_raw.csv   – wide, all years)
#          └── processed/<indicator>_clean.csv – tidy (Economy,Year,Value)
# -------------------------------------------------------------

from google.colab import drive
drive.mount('/content/drive', force_remount=True)

import os, re, glob, io, tempfile, zipfile, requests, pandas as pd

# ── CONFIG ────────────────────────────────────────────────
URL  = "https://kidb.adb.org/explore/download/c5feaa1a-c3c7-4d78-8aa7-52aeef9913a2"
ROOT = "/content/drive/MyDrive/ADB/Global/Electricity"

MERGED_RAW_DIR  = os.path.join(ROOT, "Merged_Indicators", "raw")
MERGED_PROC_DIR = os.path.join(ROOT, "Merged_Indicators", "processed")
IND_ROOT        = os.path.join(ROOT, "Individual_Indicators")

for d in (MERGED_RAW_DIR, MERGED_PROC_DIR, IND_ROOT):
    os.makedirs(d, exist_ok=True)

# wipe merged folders so newest files always replace the old ones
for d in (MERGED_RAW_DIR, MERGED_PROC_DIR):
    for f in glob.glob(os.path.join(d, "*")):
        try: os.remove(f)
        except: pass

def slugify(txt: str) -> str:
    """Safe folder/file name from indicator string."""
    return re.sub(r'[^\w\s-]', '', txt).strip().replace(" ", "_")

# ── 1. DOWNLOAD  ───────────────────────────────────────────
resp = requests.get(URL, timeout=60)
resp.raise_for_status()
content = resp.content

with tempfile.TemporaryDirectory() as tmp:
    if content[:2] == b"PK":                          # ZIP archive
        zpath = os.path.join(tmp, "data.zip")
        open(zpath, "wb").write(content)
        with zipfile.ZipFile(zpath) as z:
            member = next(n for n in z.namelist()
                           if n.lower().endswith((".csv", ".xlsx")))
            raw_bytes = z.read(member)
            ext = os.path.splitext(member)[1].lower()
    else:                                             # single file
        raw_bytes = content
        ext = ".csv" if b"," in content.splitlines()[0] else ".xlsx"

raw_merged_path = os.path.join(MERGED_RAW_DIR,
                               f"electricity_indicators_raw{ext}")
open(raw_merged_path, "wb").write(raw_bytes)

# ── 2. LOAD RAW DATAFRAME ─────────────────────────────────
read_fn = pd.read_csv if ext == ".csv" else pd.read_excel
df = read_fn(io.BytesIO(raw_bytes))

df.columns = [c.strip().replace("\n", " ") for c in df.columns]
assert "Indicator" in df.columns and "Economy" in df.columns, \
    "Expected columns 'Indicator' and 'Economy' not found!"

# ── 3. LONG → WIDE (8‑column merged table) ───────────────
year_cols = [c for c in df.columns if re.fullmatch(r"\d{4}", str(c))]
long_df = (df
           .melt(id_vars=["Indicator", "Economy"],
                 value_vars=year_cols,
                 var_name="Year",
                 value_name="Value")
           .dropna(subset=["Value"]))

pivot_df = (long_df
            .pivot_table(index=["Economy", "Year"],
                         columns="Indicator",
                         values="Value",
                         aggfunc="first")
            .reset_index())

pivot_df.columns.name = None  # drop MultiIndex name

# expected six indicators (adjust list if ADB adds more later)
expected = [
    "Total Electricity Production (kWh billion)",
    "Sources of Electricity, Combustible Fuels (% of total)",
    "Sources of Electricity, Hydropower (% of total)",
    "Sources of Electricity, Solar (% of total)",
    "Sources of Electricity, Others (% of total)",
    "Electric Power Consumption (kWh per capita)",
]

# keep only those six, silently ignore any missing / extra cols
cols_present = [c for c in expected if c in pivot_df.columns]
pivot_df = pivot_df[["Economy", "Year"] + cols_present]

# save processed merged CSV
processed_merged_path = os.path.join(
    MERGED_PROC_DIR, "electricity_indicators_merged_pivot.csv"
)
pivot_df.to_csv(processed_merged_path, index=False)

# ── 4. INDIVIDUAL INDICATOR FILES ─────────────────────────
for ind_name, grp in df.groupby("Indicator"):
    slug = slugify(ind_name)

    # create/clean folders
    raw_dir  = os.path.join(IND_ROOT, slug, "raw")
    proc_dir = os.path.join(IND_ROOT, slug, "processed")
    os.makedirs(raw_dir,  exist_ok=True)
    os.makedirs(proc_dir, exist_ok=True)
    for p in glob.glob(os.path.join(raw_dir, "*")):  os.remove(p)
    for p in glob.glob(os.path.join(proc_dir, "*")): os.remove(p)

    # wide raw (all year columns)
    grp.to_csv(os.path.join(raw_dir, f"{slug}_raw.csv"), index=False)

    # tidy clean (Economy‑Year‑Value)
    tidy = (long_df[long_df["Indicator"] == ind_name]
            .drop(columns="Indicator"))
    tidy.to_csv(os.path.join(proc_dir, f"{slug}_clean.csv"), index=False)

# ── 5. SHOW SAMPLE ────────────────────────────────────────
print("\nSample of the merged 8‑column dataset:")
print(pivot_df.head())


ValueError: mount failed

ENERGY

In [2]:
# -------------------------------------------------------------
#  Google Colab single‑cell script   (ADB › Energy indicators)
# -------------------------------------------------------------
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

import os, re, glob, io, tempfile, zipfile, requests, pandas as pd

# ── CONFIG ───────────────────────────────────────────────────
URL  = "https://kidb.adb.org/explore/download/4e61528b-8aec-4c49-951c-b36641766eeb"
ROOT = "/content/drive/MyDrive/ADB/Global/Energy"

MERGED_RAW_DIR  = os.path.join(ROOT, "Merged_Indicators", "raw")
MERGED_PROC_DIR = os.path.join(ROOT, "Merged_Indicators", "processed")
IND_ROOT        = os.path.join(ROOT, "Individual_Indicators")

for d in (MERGED_RAW_DIR, MERGED_PROC_DIR, IND_ROOT):
    os.makedirs(d, exist_ok=True)

# Wipe previous merged files so each run keeps only the latest pull
for d in (MERGED_RAW_DIR, MERGED_PROC_DIR):
    for f in glob.glob(os.path.join(d, "*")):
        try: os.remove(f)
        except: pass

def slugify(txt: str) -> str:
    return re.sub(r'[^\w\s-]', '', txt).strip().replace(" ", "_")

# ── 1. DOWNLOAD  ─────────────────────────────────────────────
resp = requests.get(URL, timeout=60)
resp.raise_for_status()
content = resp.content

with tempfile.TemporaryDirectory() as tmp:
    if content[:2] == b"PK":                               # ZIP archive
        zpath = os.path.join(tmp, "data.zip"); open(zpath, "wb").write(content)
        with zipfile.ZipFile(zpath) as z:
            member = next(m for m in z.namelist() if m.lower().endswith((".csv", ".xlsx")))
            raw_bytes = z.read(member)
            ext = os.path.splitext(member)[1].lower()
    else:                                                  # single file
        raw_bytes = content
        ext = ".csv" if b"," in content.splitlines()[0] else ".xlsx"

raw_merged_path = os.path.join(MERGED_RAW_DIR, f"energy_indicators_raw{ext}")
open(raw_merged_path, "wb").write(raw_bytes)

# ── 2. LOAD RAW DATAFRAME ───────────────────────────────────
read_fn = pd.read_csv if ext == ".csv" else pd.read_excel
df = read_fn(io.BytesIO(raw_bytes))

df.columns = [c.strip().replace("\n", " ") for c in df.columns]
assert {"Indicator", "Economy"}.issubset(df.columns), "Columns 'Indicator' & 'Economy' required!"

# ── 3. LONG → WIDE MERGED TABLE (Economy, Year, indicators) ─
year_cols = [c for c in df.columns if re.fullmatch(r"\d{4}", str(c))]
long_df = (df.melt(id_vars=["Indicator", "Economy"],
                   value_vars=year_cols,
                   var_name="Year",
                   value_name="Value")
           .dropna(subset=["Value"]))

pivot_df = (long_df
            .pivot_table(index=["Economy", "Year"],
                         columns="Indicator",
                         values="Value",
                         aggfunc="first")
            .reset_index())

pivot_df.columns.name = None  # flatten

# Save processed merged CSV
proc_merged_path = os.path.join(MERGED_PROC_DIR,
                                "energy_indicators_merged_pivot.csv")
pivot_df.to_csv(proc_merged_path, index=False)

# ── 4. INDIVIDUAL INDICATOR FILES ───────────────────────────
for ind, grp in df.groupby("Indicator"):
    slug = slugify(ind)
    raw_dir  = os.path.join(IND_ROOT, slug, "raw")
    proc_dir = os.path.join(IND_ROOT, slug, "processed")
    os.makedirs(raw_dir,  exist_ok=True)
    os.makedirs(proc_dir, exist_ok=True)
    # purge previous runs
    for p in glob.glob(os.path.join(raw_dir, "*")):  os.remove(p)
    for p in glob.glob(os.path.join(proc_dir, "*")): os.remove(p)

    grp.to_csv(os.path.join(raw_dir, f"{slug}_raw.csv"), index=False)

    tidy = (long_df[long_df["Indicator"] == ind]
            .drop(columns="Indicator"))           # Economy, Year, Value
    tidy.to_csv(os.path.join(proc_dir, f"{slug}_clean.csv"), index=False)

# ── 5. SHOW SAMPLE ──────────────────────────────────────────
print("\nSample of merged Energy dataset (columns:", len(pivot_df.columns), ")")
print(pivot_df.head())
# -------------------------------------------------------------
#  Google Colab single‑cell script   (ADB › Energy indicators)
# -------------------------------------------------------------
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

import os, re, glob, io, tempfile, zipfile, requests, pandas as pd

# ── CONFIG ───────────────────────────────────────────────────
URL  = "https://kidb.adb.org/explore/download/4e61528b-8aec-4c49-951c-b36641766eeb"
ROOT = "/content/drive/MyDrive/ADB/Global/Energy"

MERGED_RAW_DIR  = os.path.join(ROOT, "Merged_Indicators", "raw")
MERGED_PROC_DIR = os.path.join(ROOT, "Merged_Indicators", "processed")
IND_ROOT        = os.path.join(ROOT, "Individual_Indicators")

for d in (MERGED_RAW_DIR, MERGED_PROC_DIR, IND_ROOT):
    os.makedirs(d, exist_ok=True)

# Wipe previous merged files so each run keeps only the latest pull
for d in (MERGED_RAW_DIR, MERGED_PROC_DIR):
    for f in glob.glob(os.path.join(d, "*")):
        try: os.remove(f)
        except: pass

def slugify(txt: str) -> str:
    return re.sub(r'[^\w\s-]', '', txt).strip().replace(" ", "_")

# ── 1. DOWNLOAD  ─────────────────────────────────────────────
resp = requests.get(URL, timeout=60)
resp.raise_for_status()
content = resp.content

with tempfile.TemporaryDirectory() as tmp:
    if content[:2] == b"PK":                               # ZIP archive
        zpath = os.path.join(tmp, "data.zip"); open(zpath, "wb").write(content)
        with zipfile.ZipFile(zpath) as z:
            member = next(m for m in z.namelist() if m.lower().endswith((".csv", ".xlsx")))
            raw_bytes = z.read(member)
            ext = os.path.splitext(member)[1].lower()
    else:                                                  # single file
        raw_bytes = content
        ext = ".csv" if b"," in content.splitlines()[0] else ".xlsx"

raw_merged_path = os.path.join(MERGED_RAW_DIR, f"energy_indicators_raw{ext}")
open(raw_merged_path, "wb").write(raw_bytes)

# ── 2. LOAD RAW DATAFRAME ───────────────────────────────────
read_fn = pd.read_csv if ext == ".csv" else pd.read_excel
df = read_fn(io.BytesIO(raw_bytes))

df.columns = [c.strip().replace("\n", " ") for c in df.columns]
assert {"Indicator", "Economy"}.issubset(df.columns), "Columns 'Indicator' & 'Economy' required!"

# ── 3. LONG → WIDE MERGED TABLE (Economy, Year, indicators) ─
year_cols = [c for c in df.columns if re.fullmatch(r"\d{4}", str(c))]
long_df = (df.melt(id_vars=["Indicator", "Economy"],
                   value_vars=year_cols,
                   var_name="Year",
                   value_name="Value")
           .dropna(subset=["Value"]))

pivot_df = (long_df
            .pivot_table(index=["Economy", "Year"],
                         columns="Indicator",
                         values="Value",
                         aggfunc="first")
            .reset_index())

pivot_df.columns.name = None  # flatten

# Save processed merged CSV
proc_merged_path = os.path.join(MERGED_PROC_DIR,
                                "energy_indicators_merged_pivot.csv")
pivot_df.to_csv(proc_merged_path, index=False)

# ── 4. INDIVIDUAL INDICATOR FILES ───────────────────────────
for ind, grp in df.groupby("Indicator"):
    slug = slugify(ind)
    raw_dir  = os.path.join(IND_ROOT, slug, "raw")
    proc_dir = os.path.join(IND_ROOT, slug, "processed")
    os.makedirs(raw_dir,  exist_ok=True)
    os.makedirs(proc_dir, exist_ok=True)
    # purge previous runs
    for p in glob.glob(os.path.join(raw_dir, "*")):  os.remove(p)
    for p in glob.glob(os.path.join(proc_dir, "*")): os.remove(p)

    grp.to_csv(os.path.join(raw_dir, f"{slug}_raw.csv"), index=False)

    tidy = (long_df[long_df["Indicator"] == ind]
            .drop(columns="Indicator"))           # Economy, Year, Value
    tidy.to_csv(os.path.join(proc_dir, f"{slug}_clean.csv"), index=False)

# ── 5. SHOW SAMPLE ──────────────────────────────────────────
print("\nSample of merged Energy dataset (columns:", len(pivot_df.columns), ")")
print(pivot_df.head())


Mounted at /content/drive



Sample of merged Energy dataset (columns: 18 )
     Economy  Year Coal consumption Coal exports Coal imports Coal production  \
0  Australia  2015         55913344  392348681.2       125996     445761929.3   
1  Australia  2016         59261030  389300203.7        31282     437041499.8   
2  Australia  2017         61140641  378938582.9       103900     443001999.2   
3  Australia  2018         62210989  381944255.6       228167     449938264.5   
4  Australia  2019         60961575  393297916.1       181460     460453830.1   

  Crude petroleum consumption Crude petroleum exports Crude petroleum imports  \
0                 30227667.91             13030995.38                20875382   
1                 26410241.71             11946488.24             16976615.86   
2                 25629597.78             11011318.16             17511063.72   
3                 27075260.34             11209573.42             19256063.92   
4                 26999317.38             12675735.37       

Demographics

In [3]:
# -------------------------------------------------------------
#  Google Colab single‑cell script   (ADB › Demographics)
# -------------------------------------------------------------
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

import os, re, glob, io, tempfile, zipfile, requests, pandas as pd

# ── CONFIG ───────────────────────────────────────────────────
URL  = "https://kidb.adb.org/explore/download/415c4ae7-64e4-489d-abc3-1797a2febc46"
ROOT = "/content/drive/MyDrive/ADB/Global/Demographics"

MERGED_RAW_DIR  = os.path.join(ROOT, "Merged_Indicators", "raw")
MERGED_PROC_DIR = os.path.join(ROOT, "Merged_Indicators", "processed")
IND_ROOT        = os.path.join(ROOT, "Individual_Indicators")

for d in (MERGED_RAW_DIR, MERGED_PROC_DIR, IND_ROOT):
    os.makedirs(d, exist_ok=True)

# wipe previous merged files so each run keeps only the latest pull
for d in (MERGED_RAW_DIR, MERGED_PROC_DIR):
    for f in glob.glob(os.path.join(d, "*")):
        try: os.remove(f)
        except: pass

def slugify(txt: str) -> str:
    return re.sub(r'[^\w\s-]', '', txt).strip().replace(" ", "_")

# ── 1. DOWNLOAD DATASET ─────────────────────────────────────
resp = requests.get(URL, timeout=60)
resp.raise_for_status()
content = resp.content

with tempfile.TemporaryDirectory() as tmp:
    if content[:2] == b"PK":                               # ZIP archive
        zpath = os.path.join(tmp, "data.zip")
        open(zpath, "wb").write(content)
        with zipfile.ZipFile(zpath) as z:
            member = next(m for m in z.namelist()
                           if m.lower().endswith((".csv", ".xlsx")))
            raw_bytes = z.read(member)
            ext = os.path.splitext(member)[1].lower()
    else:                                                  # single file
        raw_bytes = content
        ext = ".csv" if b"," in content.splitlines()[0] else ".xlsx"

raw_merged_path = os.path.join(MERGED_RAW_DIR,
                               f"demographics_indicators_raw{ext}")
open(raw_merged_path, "wb").write(raw_bytes)

# ── 2. LOAD RAW DATAFRAME ───────────────────────────────────
read_fn = pd.read_csv if ext == ".csv" else pd.read_excel
df = read_fn(io.BytesIO(raw_bytes))

df.columns = [c.strip().replace("\n", " ") for c in df.columns]
assert {"Indicator", "Economy"}.issubset(df.columns), \
    "Columns 'Indicator' & 'Economy' required!"

# ── 3. PIVOT TO MERGED TABLE  (Economy · Year · indicators) ─
year_cols = [c for c in df.columns if re.fullmatch(r"\d{4}", str(c))]
long_df = (df.melt(id_vars=["Indicator", "Economy"],
                   value_vars=year_cols,
                   var_name="Year",
                   value_name="Value")
           .dropna(subset=["Value"]))

pivot_df = (long_df
            .pivot_table(index=["Economy", "Year"],
                         columns="Indicator",
                         values="Value",
                         aggfunc="first")
            .reset_index())

pivot_df.columns.name = None  # flatten the header

proc_merged_path = os.path.join(
    MERGED_PROC_DIR, "demographics_indicators_merged_pivot.csv"
)
pivot_df.to_csv(proc_merged_path, index=False)

# ── 4. SPLIT INTO INDIVIDUAL INDICATOR FOLDERS ─────────────
for ind_name, grp in df.groupby("Indicator"):
    slug = slugify(ind_name)
    raw_dir  = os.path.join(IND_ROOT, slug, "raw")
    proc_dir = os.path.join(IND_ROOT, slug, "processed")
    os.makedirs(raw_dir,  exist_ok=True)
    os.makedirs(proc_dir, exist_ok=True)

    # clear previous run
    for p in glob.glob(os.path.join(raw_dir, "*")):  os.remove(p)
    for p in glob.glob(os.path.join(proc_dir, "*")): os.remove(p)

    # wide raw file (all year columns)
    grp.to_csv(os.path.join(raw_dir, f"{slug}_raw.csv"), index=False)

    # tidy clean file: Economy, Year, Value
    tidy = (long_df[long_df["Indicator"] == ind_name]
            .drop(columns="Indicator"))
    tidy.to_csv(os.path.join(proc_dir, f"{slug}_clean.csv"), index=False)

# ── 5. SHOW SAMPLE ──────────────────────────────────────────
print("\nSample of merged Demographics dataset (columns:", len(pivot_df.columns), ")")
print(pivot_df.head())


Mounted at /content/drive



Sample of merged Demographics dataset (columns: 13 )
     Economy  Year Age Dependency Ratio Gini Coefficient  \
0  Australia  2015          51.06329837              ...   
1  Australia  2016          51.60344566      0.336858104   
2  Australia  2017          52.04242094              ...   
3  Australia  2018           52.4455123      0.343337622   
4  Australia  2019          52.91976024              ...   

  Growth Rates in Population Human Development Index  \
0                1.428871347                   0.933   
1                1.549756313                   0.936   
2                1.633418183                   0.937   
3                1.484979951                   0.941   
4                1.466362474                   0.941   

  Income Ratio of Highest 20% to Lowest 20%  \
0                                       ...   
1                               5.554054054   
2                                       ...   
3                               5.726027397   
4            